# Practical 11-2: Informed Search

## 1.1 The 8-Puzzle Problem

▪ **8-puzzle** is a sliding puzzle that consists of a frame of 8 square tiles in random order with one tile missing. 

<img src="child_nodes.png" width="600">

▪ It is a classical problem for modelling algorithms involving **heuristics** (or additional information about the estimate distance from the current state to the goal).

### 1.1.1 Solving an 8-Puzzle Problem

▪ Provided an initial state, which could be a random configuration of the positions of the tiles, the **empty space** has to be moved one step at a time to arrive to state equals to the goal state.

![](8puzzle.png)

### 1.1.2 Problem Solving Steps

▪ **Step 1**: Get the current state.

▪ **Step 2**: Find the available moves and their cost.

▪ **Step 3**: Choose the move with the least cost and set it as the current state.

▪ **Step 4**: Check if it matches the goal state. If yes, then terminate, else, move to step 1.

### 1.1.3 Solution: State Representation

▪ To setup the eight-puzzle, we need to formulate its' state space in terms of: 1) **StartNode**, 2) **GoalNode**, 3) **PreviousNode**, 4) **Fringe**, and 5) **Parent**.

▪ All of these can be represented using the attributes of an object (e.g., an object of class Puzzle).

▪ A single dimension (1D) list can be used to represent the states instead of using a two dimensional (2D) list where '0' indicates empty space. 

▪ **StartNode** represents the actual puzzle as shown below:

                                The default StartNode = ['1','2','3','4','5','6','0','7','8']

<img src="1d.jpg" width="150">

In [ ]:
class Puzzle:
    def __init__(self):
        self.start_node = ['1', '2', '3', '4', '5', '6', '0', '7', '8'] # Default, h(n) = 8
        self.goal_node = ['1', '2', '3', '4', '5', '6', '7', '8', '0']  # default, h(n) = 0
        self.previous_node = []  # To store the expanded nodes
        self.fringe = []         # To store the leaf nodes (or to be expanded nodes)
        self.parent = []

### 1.1.4 Solution: The Successor Function

▪ The states (or next move) are generated with the use of the **successor** function.

<img src="successors.jpg" width="600">

▪ Four if-blocks are used to generated all possible successors depending on the location of the tile '0'.

▪ For instance, if '0' is at top-most and left-most location, then there will be 2 successors, where as if '0' is at the bottom-most and middle location, then there will be 3 successors.

In [ ]:
    # Successor function where self refers to the object that calls this function
    def successor(self, node = [], search = 'sahc'):

        sub_node = [] # It is a list to store a child node at a later stage temporarily 
        p = [] 
        c = [] 
        
        # To get the location of the empty tile '0' which begins from 1 and end at 9
        get_zero_location = node.index('0') + 1 
        
        # To join node to the end of sub_node with node 
        sub_node.extend(node[0:-1]) 
        
        # To determine the left/right boundaries of the empty tile '0'
        boundary = self.boundaries(get_zero_location)
        
        # To join node list to the end of p with the entire node
        p.extend(node)
        
        if search == 'sahc':
            # Without this line turns the search to best first search because SAHC only looks at the surrounding nodes
            self.fringe = [] 
        else:
            pass
        
        # If the empty tile is at the top 2 rows, then move the empty tile DOWN
        if get_zero_location + 3 <= 9:
            c = [] 
            
            # Swap the location of 2 tiles
            temp = sub_node[node.index('0')] 
            sub_node[node.index('0')] = sub_node[node.index('0') + 3] 
            sub_node[node.index('0') + 3] = temp 
            
            # To join node to the end of c with node (excluding its last item)
            c.extend(self.heuristic(sub_node))
            
            # Add the child node c to the fringe (which is also the frontier) to be expanded later 
            self.fringe.append(c)
            
            # Reprocess sub_node before it is processed in the following if-block
            sub_node = [] 
            sub_node.extend(node[0:-1])
            
            # Create a tuple with a pair of child and parent
            cp = (c, p)           
            if (cp not in self.parent):
                self.parent.append(cp)
        
        # If the empty tile is at the bottom 2 rows, then move the empty tile UP
        if get_zero_location - 3 >= 1: 
            c = []
            
            temp = sub_node[node.index('0')]
            sub_node[node.index('0')] = sub_node[node.index('0') - 3]
            sub_node[node.index('0') - 3] = temp
            
            c.extend(self.heuristic(sub_node))            
            self.fringe.append(c)
            
            sub_node = []
            sub_node.extend(node[0:-1])
            
            cp = (c, p)            
            if (cp not in self.parent):
                self.parent.append(cp)
        
        # If the empty tile is on the last 2 columns, then move the empty tile LEFT
        if get_zero_location - 1 >= boundary[0]: 
            c = []
            
            temp = sub_node[node.index('0')]
            sub_node[node.index('0')] = sub_node[node.index('0') - 1]
            sub_node[node.index('0') - 1] = temp
            
            c.extend(self.heuristic(sub_node))
            self.fringe.append(c)
            
            sub_node = []
            sub_node.extend(node[0:-1])
            
            cp = (c,p)
            if(cp not in self.parent):
                self.parent.append(cp)
        
        # If the empty tile is on the first 2 columns, then move the empty tile RIGHT
        if get_zero_location + 1 <= boundary[1]: 
            c = []
            
            temp = sub_node[node.index('0')]
            sub_node[node.index('0')] = sub_node[node.index('0') + 1]
            sub_node[node.index('0') + 1] = temp
            
            c.extend(self.heuristic(sub_node))
            self.fringe.append(c)
            
            sub_node = []
            sub_node.extend(node[0:-1])
            
            cp = (c,p)
            if(cp not in self.parent):
                self.parent.append(cp)

### 1.1.5 Solution: The Boundaries Function

▪ The **boundaries** function can be used to find the left and right boundary of '0' which helps in identifying relevant moves (e.g., left or right).

▪ For instance, if '0' is at middle or right column, then **left** is one possible move (or successor). 

In [ ]:
    # location holds the location of the empty tile '0'
    def boundaries(self, location): 
        
        # location of each tile in the puzzle (assumption: the first location = 1 but not 0)
        rows = [[1, 2, 3], [4, 5, 6], [7, 8, 9]] 
        
        low = 0
        high = 0
        
        # To determine the left/right boundaries of the empty tile '0' 
        for i in rows:  
            if location in i: 
                low = i[0] 
                high = i[2] 
                break
        
        return [low, high]

### 1.1.6 Solution: The Heuristic Function

▪ In this example, a simple heuristic is used to calculate the cost of a possible move.

▪ The heuristic function is implemented by counting the number of misplaced tiles.

▪ Therefore, a goal state should have heuristic cost equals to 0, and a state with a lower heuristic cost is a better option than a state with higher heuristics cost.

In [ ]:
    # The heuristic function is used to calculate the number of misplaced_tile as the cost
    def heuristic(self, node):
        misplaced_tile = 0
        
        for i in range(9):
            # Compare the tile between the current node and the goal_node location by location
            if node[i] != self.goal_node[i]:
                # If the tile is misplaced, increase the value one
                misplaced_tile += 1 
        
        # Append the heuristic cost to the last place of node
        node.append(misplaced_tile) 
        
        return node

### 1.1.7 Solution: The PrintSolution Function

▪ The **print_solution()** function prints the solution path from the start node to the goal node.

In [ ]:
    # Functions to display output by showing how to yield the solution by observing the expanded nodes
    def print_solution_steps(self):
        count = 1
        for i in self.previous_node:
            print("step ", count, i[0:-1], "(", i[-1], ")")
            count += 1
        
    # To show the final path
    def print_solution(self):
        goal = self.goal_node
        path_list = [goal] 
        
        self.parent.reverse()      
        
        # This loop will append parent of a node to path_list beginning from goal until root
        for i in self.parent:  
            child, parent = i  
            
            # If the first node is the goal node
            if(child == goal): 
                goal = parent 
                path_list.append(goal) 
            # Stop when reach the start_node (the end of the list)    
            if parent == self.start_node:
                break
        
        # Reverse the path list so that it begins with the start_node
        path_list.reverse() 
        
        count = 0
        for node in path_list:
            print("LEVEL", count, ":", node[0:-1], "(", node[-1], ")")
            count += 1

### 1.1.8 Solution: Putting All Together

▪ The successor function generates all possible next moves. 

▪ The boundaries function determines what possible moves can be generated.

▪ The heuristic function determines the quality of a possible move.

▪ The print_solution() function generates the solution path.

▪ Search algorithms are needed to look for an optimal solution for the 8-puzzle problem:

\>>> **Steepest ascent hill climbing search** or **Hill Climbing**

\>>> **Greedy-best first search**

## 1.2 Steepest Ascent Hill Climbing (SAHC)

▪ **Steepest ascent hill climbing** is a local search strategy.

▪ It tries to improve the current state by looking for the best immediate neighbors as the next move.

<img src="ashc.png" width="700">

▪ Thus, unlike greedy best first search, it does not have to maintain a search tree or the complete set frontiers.

### 1.2.1 The Steepest Ascent Hill Climbing Algorithm

▪ The **steepest_ascent_hill_climbing function** controls the loop of the generation of successors until a goal node is found.

▪ The **SAHC function** compares the heuristic cost of the current node with the heuristic cost of all successors to identify the best one as the next move. 

▪ If the heuristic cost equals 0, then a goal node has been found.

![](hillclimbing.png)

### 1.2.2 Implementation: steepest_ascent_hill_climbing()

In [ ]:
    def steepest_ascent_hill_climbing(self):
        print("STEEPEST ASCENT HILL CLIMBING\n" + "_"*60 + "\n")
        level = 0
        
        # The following two lines will add the heuristic cost to the end of the list
        current_node = self.heuristic(self.start_node) 
        goal_node = self.heuristic(self.goal_node)
        
        # Check if the goal state is achieved or not, if not, generate child nodes from the current node
        if current_node != goal_node:
            print("---------", "\nLEVEL", level, "\n---------")
            self.successor(current_node) 
            level += 1 
  
        print("CURRENT NODE:")
        print(current_node)
        print("\nOPEN LIST:")
        for i in self.fringe:
            print(i) 
        
        # Pass the current heuristic cost as the argument by accessing the last element of the node
        nx_node = self.SAHC(current_node[-1])
        
        # This loop begins from the child node of the ultimate first node, until either a goal node is found or no solution is found         
        while nx_node != [] and current_node != goalNode and current_node != nx_node:
            current_node = nx_node
            
            print("\n---------", "\nLEVEL", level, "\n---------")
            self.successor(current_node)
            level += 1
            
            print("CURRENT NODE:")
            print(current_node)
            print("\nOPEN LIST:")
            for i in self.fringe:
                print(i)
            
            nx_node = self.SAHC(current_node[-1])
            
            # Goal node is found, break out of the loop to stop generating child node
            if nx_node[-1] == 0:
                break        

        if nx_node != []:
            print()
            print("-" * 63)
            print("Goal Path (The best path)")
            print("-" * 63)
            self.print_solution()
        else:
            print("LOCAL MAXIMUM: There is no solution")

### 1.2.3 Implementation: SAHC()

In [ ]:
    # This function will return the best move as the next move (from all choices in the frontier)
    def SAHC(self, hr_cost):
        nx_node = [] 
        is_better = False
        
        # Loop through frontier to search for the best successor based on hr_cost (the lower the better)
        while True:   
            for i in self.fringe: 
                if (i[-1] < hr_cost): 
                    hr_cost = i[-1]  
                    nx_node = i   
                    is_better = True 
            
            # if the best successor is chosen but it has already been chosen in the past...
            if nx_node in self.previous_node and nx_node in self.fringe:
                self.fringe.remove(nx_node) 
            # if the best successor is available and it is not in the previous_node list
            elif nx_node != []:
                if nx_node[-1] == 0:
                    print("\nSolution:", nx_node)
                else:
                    print("\nSelected:", nx_node)
                
                self.previous_node.append(nx_node)
                
                return nx_node 
            # if the best successor is not available            
            else:
                if is_better == False:
                    print("\nThere is no better move")
                return nx_node

### 1.2.5 Execution of Search Strategy

▪ Now let’s try to run the code as in the EightPuzzle.py. 

▪ Before it can be executed, we need to first import the module.

▪ The code has a default configuration for the StartNode, **random.shuffle()** can be used to reshuffle the order of the tiles.

### 1.2.6 Test Run \#1

▪ StartNode = ['1', '2', '3', '4', '5', '6', '0', '7', '8'] # Default, h(n) = 3

▪ GoalNode = ['1', '2', '3', '4', '5', '6', '7', '8', '0']  # default, h(n) = 0

▪ Import a specific class (e.g., Puzzle) from one file using the import command

https://www.entechin.com/how-to-import-a-class-from-another-file-in-python/

In [ ]:
import random
from EightPuzzle import Puzzle

puzzle8 = Puzzle()
puzzle8.steepestAscentHillClimbing()

### 1.2.7 Test Run \#1: Explanation

▪ Import multiple classes from one file using the import command (e.g., assuming that there are two classes in EightPuzzle.py)

https://www.entechin.com/how-to-import-a-class-from-another-file-in-python/

In [ ]:
import random
import EightPuzzle

p8 = EightPuzzle.Puzzle()
p8.steepestAscentHillClimbing()

### 1.2.8 Test Run \#2

▪ StartNode = any randomly generated configuration

▪ GoalNode = ['1', '2', '3', '4', '5', '6', '7', '8', '0']  # default, h(n) = 0

▪ Expected outcome of the test run:

\>>> SAHC does not guarantee completeness as shown above

\>>> For most of the time, after expanded the current node in level 1 or 2, there will be no better move in the open list (or frontier), and hence no node is selected. 

▪ Obviously a local maximum problem exists...

In [ ]:
import random
from EightPuzzle import Puzzle

p8 = Puzzle()
random.shuffle(p8.StartNode)
p8.steepestAscentHillClimbing()

### 1.2.9 Pop Quiz: Which type of local maximum problem exists in the hill climbing search above?

![](localmaxima.png)

▪ **Shoulder** ==> Plateaux (A flat area of the state-space landscape)

▪ **Local maximum**

▪ **Flat local maximum** ==> Plateaux (A flat area of the state-space landscape)

▪ **Ridges** 

\>>> A ridge is a special kind of local maxima with sequences of local maxima not directly connected to each other.

\>>> In other words, at the peak, every neighbor appears to be downhill, but in fact the search space has an uphill moves (which is after the downhill move)

## 1.3 Best First Search

### 1.3.1 An Introduction to Best First Search

▪ Unlike SAHC, **best first search** would backtrack when the current path does not seem promising anymore. 

▪ In order to be able to do so, it has to keep track of all successors in the frontier, both immediate and non-immediate ones.

<img src="bfs.png" width="500">

### 1.3.2 Best First Search Algorithm

▪ Here is the general algorithm of a greedy-best first search:

\>>> Select node for expansion with minimal evaluation function f(n) where f(n) is some function that includes the estimate heuristic h(n) of the remaining distance to goal, i.e. f(n) = h(n)

\>>> Implement using priority queue, i.e. expand the best node in the open list (regardless the parent’s heuristic cost)

### 1.3.3 Implementation: best_first_search()

In [ ]:
    def best_first_solve(self):
        
        print("GREEDY BEST FIRST SEARCH\n" + "_"*60 +"\n")
        count = 0
        
        # The following two lines will add the heuristic cost to the end of the list
        current_node = self.heuristic(self.start_node)
        goal_node = self.heuristic(self.goal_node) # Check this!
        
        # To generate all successors, the 2nd argument will replace the default value of sahc
        self.successor(current_node,'bfs') 
        
        print("\n---------", "\nSTEP:", count, "\n---------")
        print("Current Node:", current_node)
        print("\nOpen List:", self.fringe) 
        
        nx_node = self.best_first_search()
        count += 1
        
        while nx_node != [] and nx_node != goal_node and nx_node != current_node:
            current_node = nx_node
            print("\n---------", "\nSTEP:", count, "\n---------")
            print("Current Node:", current_node)
            print("\nOpen List:", self.fringe) # to display open nodes
            self.successor(nx_node,'bfs')      # to generate next successor states
            nx_node = self.best_first_search() # search next best node
            count += 1                         # go to next step
       
        print("Final Goal:", nx_node, "\n")    # print the final goal
        print("-"*60, "\nHow the nodes are expanded\n", "-"*60)
        self.print_solution_steps()
       
        print("\n", "-"*60)
        
        print("Initial State:", self.start_node)
        print("Goal State:", self.goal_node)
        print("\nGoal Path (the best path)")
        print("-"*60)
        self.print_solution()

    # Best first seach will select child node with the lowest heuristic cost from the fringe nodes
    def best_first_search(self):
        nx_node = [] 
        while True:
            # The default value set for heuristic cost, the highest cost is still lower than this value
            hr_cost = 100000
            
            for i in self.fringe:     # For each open node in the fringe list
                if (i[-1] < hr_cost): # The heuristic cost is the last member
                    hr_cost = i[-1]   # Update heuristic cost
                    nx_node = i       # Set the best node to temporary node
            
            # If the temporary node is already in the previous_node list
            if nx_node in self.previous_node and nx_node in self.fringe:
                self.fringe.remove(nx_node) # Remove the temporary node from the open list
            # If it is not in the previous_node list  
            else:
                self.previous_node.append(nx_node) # Add the temporary node to previous_node list
                return nx_node  # Return the next node

### 1.3.4 Test Run \#1

▪ StartNode = ['1', '2', '3', '4', '5', '6', '0', '7', '8'] # Default, h(n) = 3

▪ GoalNode = ['1', '2', '3', '4', '5', '6', '7', '8', '0']  # default, h(n) = 0

In [ ]:
import random
from EightPuzzle import Puzzle

p8 = Puzzle()
p8.bestFirstSolve()

### 1.3.5 Test Run \#2

▪ StartNode = any randomly generated configuration

▪ GoalNode = ['1', '2', '3', '4', '5', '6', '7', '8', '0']  # default, h(n) = 0

In [ ]:
import random
from EightPuzzle import Puzzle

p8 = Puzzle()
random.shuffle(p8.StartNode)
p8.bestFirstSolve()

## 1.4 Exercise

### 1.4.1 Question \#1

▪ Add a **simpleHillClimbing** function to the code in **EightPuzzle.py** to get a path that reaches the goal state from the start state. 

▪ Then call and run the **simpleHillClimbing** function in **heuristic_search.py**. 

▪ Check whether the algorithm could guarantee completeness and optimality.

### 1.4.2 Question \#2

▪ Modify the code in **EightPuzzle.py** by adding the **aStar (A*)** function to the code in **EightPuzzle.py** to get the optimal solution from start node to the goal (Remark: Back up the **EightPuzzle.py** before doing any modification). 

▪ Add another function called **functionHeuristic** (i.e. f(n)) so that f(n) = h(n) + g(n), where g(n) = the cost (so far) to reach the node from the start node. 

▪ Similar to the **greedy-best first search**, however you shall append the f(n) to the node instead of h(n) as shown in the heuristic function. 

▪ Then call and run the **aStar** function in **heuristic_search.py** and observe the space complexity and time complexity of the algorithm.